In [1]:
import ifcopenshell
from ifctester import reporter
from ifctester import ids

No stream support: No module named 'lark'


# Create the IDS Renhold (IDS Cleaning)

[IDS docutmentation](https://github.com/buildingSMART/IDS/blob/master/Documentation) in buildingSMART github repo. 


## Create IDS
Start by folling the [metadata guidelines found here.](https://github.com/buildingSMART/IDS/blob/master/Documentation/ids-metadata.md)

In [2]:
# create new IDS 
# ref https://github.com/buildingSMART/IDS/blob/master/Documentation/ids-metadata.md 
my_ids = ids.Ids(
   title="IDS Renhold",
   author="sigve@mok-see.com",
   copyright="[mok-see] AS",
   version="0.5",
   purpose= "Basis for kalkulere renholdskostnader",
   description="Arkitektmodell kan bli brukt til å kalkulere renholdskostnader hvis den inneholder \
      informasjonen som er definert i denne Informasjons-leveranse-spesifikasjonen (IDS-Informaton Delivery Specification)\
         Denne IDS filen kan brukes for å automatisk validere at en IFC-modell. \
            Prosjektet er en del av pilotprosjekt i regi av DiBK",
   
   milestone="MMI 500 : Som bygget")

## Create specifications

## Requirement for naming
Rooms need to be given proper names and numbering that can be found on in the pphysical building

In [3]:
# Create naming specification
naming_spec = ids.Specification(
    name="Navngivning på rom",
    description= "Alle rom må ha et beskrivende navn og romnummer som kan gjennkjennes på merking i bygget",
    instructions="Alle rom må kunne identifiseres med romnavn og romnummer")
naming_spec.applicability.append(ids.Entity(name="IfcSpace"))

name_attr = ids.Attribute(
    name = "LongName",
    instructions="Alle rom må ha et romnavn"
)
naming_spec.requirements.append(name_attr)

number_attr = ids.Attribute(
    name = "Name",
    instructions="Alle rom må ha et romnummer"
)
naming_spec.requirements.append(number_attr)
# Add specification to ids.
my_ids.specifications.append(naming_spec)

In [4]:
# Create covering specification
covering_spec = ids.Specification(
    name="Dekkende materialer på rom",
    description= "Alle rom må ha referanse til dekkene som kler rommet",
    instructions="Alle rom må referanse til dekkene som kler rommet, som gir referanse til IfcCovering")
covering_spec.applicability.append(ids.Entity(name="IfcSpace"))

coverings = ids.Attribute(
    name = "HasCoverings",
    instructions="Alle rom må gi liste av relaterte dekkende flater"
)
covering_spec.requirements.append(coverings)
my_ids.specifications.append(covering_spec)

## Requirement for room size
The size of the room is required in order to estimate cleaning costs

In [6]:

room_size_spec = ids.Specification(
    name="Størrelser på rom",
    description= "Romstørrelser må oppgis for å kunne kalkulere renholdskostnader",
    instructions="Alle rom må oppgi nødvendige størrelser for å kunne kalkulere riktige renholdskostnader per rom",
    ifcVersion = "IFC4")
room_size_spec.applicability.append(ids.Entity(name="IfcSpace"))

netFloorArea = ids.Property(
    name="NetFloorArea", 
    propertySet="Qto_SpaceBaseQuantities", 
    datatype="IfcAreaMeasure",
    instructions="Netto gulvareal må være oppgitt per rom",
    minOccurs=1
    )
room_size_spec.requirements.append(netFloorArea)

finishCeilingHeight = ids.Property(
    name="FinishCeilingHeight", 
    propertySet="Qto_SpaceBaseQuantities", 
    datatype="IfcLengthMeasure",
    instructions="Himlingshøyde, netto høyde på rommet, må være oppgitt for hver rom",
    minOccurs=1
    )
room_size_spec.requirements.append(finishCeilingHeight)

concealedCeiling = ids.Property(
    name="ConcealedCeiling", 
    propertySet="Pset_SpaceCoveringRequirements", 
    datatype="IfcBoolean",
    instructions="Det må være oppgitt om det er åpen eller lukket himling på rommet",
    minOccurs=1
    )
room_size_spec.requirements.append(concealedCeiling)


my_ids.specifications.append(room_size_spec)



## Requirements for walls and floors and the realtion to rooms
Walls under 3 meter need to be cleaned, hence their net area and finish needs to be provided

In [7]:
## Coverings

room_covering_spec = ids.Specification(
    name="Overflater i rommet",
    description= "Overflater må oppgis for å kunne kalkulere renholdskostnader",
    instructions="Alle rom må oppgi nødvendige overflatematerialer for å kunne kalkulere riktige renholdskostnader per rom",
    ifcVersion = "IFC4")
room_covering_spec.applicability.append(ids.Entity(name="IfcSpace"))

wallCovering = ids.Property(
    name="WallCovering", 
    datatype="IfcLabel",
    propertySet="Pset_SpaceCoveringRequirements", 
    instructions="Overflater på vegger må være angitt",
    minOccurs=1
    )
room_covering_spec.requirements.append(wallCovering)

floorCovering = ids.Property(
    name="FloorCovering", 
    propertySet="Pset_SpaceCoveringRequirements",
    datatype="IfcLabel",
    instructions="Overflater på gulv må være angitt",
    minOccurs=1
    )
room_covering_spec.requirements.append(floorCovering)

ceilingCovering = ids.Property(
    name="CeilingCovering", 
    datatype="IfcLabel",
    propertySet="Pset_SpaceCoveringRequirements", 
    instructions="Overflater på vegger må være angitt",
    minOccurs=1
    )
room_covering_spec.requirements.append(ceilingCovering)
my_ids.specifications.append(room_covering_spec)




In [7]:
# Vegger på opp til 3 meter skal oppgis nettoareal på
wall_spec = ids.Specification(
    name="Veggareal for vegger under 3 meter",
    description= "Alle vegger opp til og med 3 meter skal vaskes iht. INSTA800",
    instructions="Alle vegger opp til og med 3 meter må ha oppgitt nettoareal for å kunne kalkulere renholdskostnader",
    ifcVersion = "IFC4",
    minOccurs = 1
)
wall_spec.applicability.append(ids.Entity(name="IfcWall"))
#wall_spec.applicability.append(ids.Entity("IfcWallStandardCase"))
wall_value_restriction = ids.Restriction(options={ "maxInclusive": 3})
wall_spec.applicability.append(ids.Property(
    propertySet="Qto_WallBaseQuantities",
    name="Height",
    measure="IfcLengthMeasure",
    value=wall_value_restriction
    )
)

netWallArea = ids.Property(
    name="NetSideArea", 
    propertySet="Qto_WallBaseQuantities", 
    measure="IfcAreaMeasure",
    instructions="Alle vegger på 3 m eller mindre må ha oppgitt netto areal",
    minOccurs=1
    )
wall_spec.requirements.append(netWallArea)

wall_material = ids.Material(minOccurs=1 ,instructions="Walls must have materials")
wall_spec.requirements.append(wall_material)

my_ids.specifications.append(wall_spec)

my_ids.specifications.append(wall_spec)




In [8]:
# lager fil
my_ids.to_xml("../ids/renholds.ids")

True

In [10]:
# open  IFC file:
my_ifc = ifcopenshell.open("../ifc/1170101-00-000-A-20_1st.ifc")
testids = ids.open("../ids/renholds.ids")
# validate IFC model against IDS requirements:
testids.validate(my_ifc)
# show results:
reporter.Json(testids).report()

{'title': 'IDS Renhold',
 'date': '2023-10-05 15:48:01',
 'specifications': [{'name': 'Navngivning på rom',
   'description': 'Alle rom må ha et beskrivende navn og romnummer som kan gjennkjennes på merking i bygget',
   'instructions': 'Alle rom må kunne identifiseres med romnavn og romnummer',
   'status': True,
   'total_applicable': 258,
   'total_applicable_pass': 258,
   'total_applicable_fail': 0,
   'percent_applicable_pass': 100,
   'total_checks': 516,
   'total_checks_pass': 516,
   'total_checks_fail': 0,
   'percent_checks_pass': 100,
   'required': False,
   'applicability': ['All IfcSpace data'],
   'requirements': [{'description': 'The LongName shall be provided',
     'status': True,
     'failed_entities': [],
     'total_applicable': 258,
     'total_pass': 258,
     'total_fail': 0},
    {'description': 'The Name shall be provided',
     'status': True,
     'failed_entities': [],
     'total_applicable': 258,
     'total_pass': 258,
     'total_fail': 0}]},
  {'nam

In [5]:
if not bool(0):
    print("yeo")

yeo
